In [1]:
from fewshot_cxray.config import Config

from fewshot_cxray.utils import *
from fewshot_cxray.loss import *
from fewshot_cxray.model import *
from fewshot_cxray.dataset import *

In [2]:
device_id='cuda:0'

resize = 2048
device = torch.device(device_id if torch.cuda.is_available() else "cpu")
device

device(type='cuda', index=0)

In [3]:
cfg = Config()

cfg.batch_size = 64

In [4]:
dataset = build_dataset(cfg=cfg, out_dir='C:\\Users\\saura\\Notebooks\\output')

print(len(dataset))

val_dataset = build_dataset(mode='val', cfg=cfg, out_dir='C:\\Users\\saura\\Notebooks\\output')

len(val_dataset)

test_dataset = build_dataset(mode='test', cfg=cfg,out_dir='C:\\Users\\saura\\Notebooks\\output')

image_datasets = {
    'train': dataset, 'val': val_dataset, 'test': test_dataset
}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=cfg.batch_size,
                                             shuffle=True, num_workers=1)
              for x in ['train', 'val', 'test']}

193730


In [5]:
with open(os.path.join('G:\\smahindr\\mimic_novel_labels.pkl'), 'rb') as f:
    dt = pickle.load(f)

novel_datasets = pd.DataFrame.from_dict(dt['label'], orient='index')

novel_datasets = novel_datasets[novel_datasets.index.isin(list(dt['image'].keys()))]

n_way = 5
n_shots = 200

import random
sampled_classes = random.sample([14,15,16,17,18], 5)

num_episodes = 100

novel_datasets = novel_datasets[~novel_datasets.index.duplicated()]


In [6]:
train_mean_norm = torch.load('train_mean_normal_baseline.pt',
                              map_location=device)
train_mean_norm = train_mean_norm.to(device)

In [7]:

train_mean_norm4 = torch.load('train_mean_tukey.pt', map_location=device)
train_mean_norm4 = train_mean_norm4.to(device)

In [8]:
device_id = 'cuda:0'

In [9]:
pthl2 = 'D:\\MIMIC_CXR\\models\\residual_network_clip_2048_12.pth'

model_finetune = ImageResNet(block=BasicBlock, 
             layers=[2, 2, 2, 2, 2, 2, 2],
                output_channels = 14, zero_init_residual=False, groups=1, 
             width_per_group=64, replace_stride_with_dilation=None, norm_layer=None)

model_finetune, _, _ = load_checkpoint(model_finetune, optim.SGD(model_finetune.parameters(), lr=0.0001, momentum=0.9), 
                                      pthl2
                                      ) 
num_ftrs = model_finetune.fc1.in_features
model_finetune.fc1 = nn.Linear(num_ftrs, 5)
model_finetune.fc1.weight.requires_grad = True
model_finetune.fc1.bias.requires_grad = True

# models.append(model_finetune2)

In [10]:
pthrcna = 'model_nca_june_13_added_unmatched_neg.pth'
model_rcna = ImageResNet(block=BasicBlock, 
             layers=[2, 2, 2, 2, 2, 2, 2],
                output_channels = 14, zero_init_residual=False, groups=1, 
             width_per_group=64, replace_stride_with_dilation=None, norm_layer=None)

model_rcna, _, _ = load_checkpoint(model_rcna, optim.SGD(model_rcna.parameters(), lr=0.0001, momentum=0.9), 
                                      pthrcna
                                      ) 
model_rcna = model_rcna.to(device)
model_rcna.eval()
pass


In [11]:
import pickle
def save_pickle(file, data):
    with open(file, 'wb') as f:
        pickle.dump(data, f)

def load_pickle(file):
    with open(file, 'rb') as f:
        return pickle.load(f)

def distribution_calibration(query, base_means, base_cov, k,alpha=0.21):
    dist = []
    for i in range(len(base_means)):
        dist.append(np.linalg.norm(query-base_means[i]))
    index = np.argpartition(dist, k)[:k]
    mean = np.concatenate([np.array(base_means)[index], query[np.newaxis, :]])
    calibrated_mean = np.mean(mean, axis=0)
    calibrated_cov = np.mean(np.array(base_cov)[index], axis=0)+alpha

    return calibrated_mean, calibrated_cov

In [12]:
import collections

In [13]:
base_means = []
base_cov = []
datatype = 'base'
base_features_path = '%s_features_12_4.plk'%datatype
with open(base_features_path, 'rb') as f:
    data = pickle.load(f)
    for key in data.keys():
        feature = np.array(data[key])
        mean = np.mean(feature, axis=0)
        cov = np.cov(feature.T)
        base_means.append(mean)
        base_cov.append(cov)

In [14]:
model_finetune = model_finetune.to(device)

In [16]:
cfg.batch_size

64

In [17]:

beta=0.5

In [18]:
from collections import defaultdict

In [19]:
class FewShotNCALossMultiLabel(torch.nn.Module):
    def __init__(
        self,
        classes=14,
        temperature=1,
        batch_size=16,
        num_labels=14,
        frac_negative_samples=1,
        frac_positive_samples=1,
    ):
        super(FewShotNCALossMultiLabel, self).__init__()
        self.temperature = torch.tensor(float(temperature), requires_grad=True).to(device)
        self.num_labels=num_labels

    def forward(self, pred, target):
        n, d = pred.shape

        p_norm = torch.pow(torch.cdist(pred, pred), 2)
        p_norm[p_norm < 1e-10] = 1e-10
        dist = torch.exp(-1 * p_norm / self.temperature).to(device)        

        all_pos = torch.zeros(n,n).to(device)
        all_negs = torch.zeros(n,n).to(device)
        negated_target = torch.tensor(~(target.type(torch.bool)),  dtype=torch.int16)

        for i in range(self.num_labels):
            positives = (target[:, i, None]*target[:, i, None].T)
#             weighted_positives = positives*pos_weights[i]
#             negatives = (negated_target[:, i, None]*negated_target[:, i, None].T)
#             unmatched = (target[:, i, None]*negated_target[:, i, None].T)#*pos_weights[i]
#             unmatched_neg = (negated_target[:, i, None]*target[:, i, None].T)
            all_pos+= positives #weighted_positives
#             all_negs += negatives
#             all_negs += unmatched
#             all_negs += unmatched_neg
#             all_negs += positives

        negatives_matrix = torch.tensor(~all_pos.type(torch.bool), dtype=torch.int16).to(device)
#         negatives_matrix = torch.tensor(all_negs, dtype=torch.int16).to(device)

        positives_matrix = (
                    torch.tensor(all_pos, dtype=torch.int16)
                ).to(device)

        positives_matrix.fill_diagonal_(0)
#         print(positives_matrix[0, :])
#         print(negatives_matrix[0, :])
#         print(dist[0, :])
        numerators = torch.sum(dist * positives_matrix, axis=0)
        denominators = torch.sum(dist* negatives_matrix, axis=0)

        denominators[denominators < 1e-10] = 1e-10
        frac = numerators / (numerators + denominators )

        loss = -1 * torch.sum(torch.log(frac[frac >= 1e-10])) / (n)   
        return loss

In [20]:
novelNCA = FewShotNCALossMultiLabel(num_labels=5)

In [21]:
class LgRegv(torch.nn.Module):
    '''
    TODO: pre-training
    from power to voronoi
    '''
    def __init__(self, dim, nla):
        super(LgRegv, self).__init__()
        self.linear = nn.Linear(dim, nla, bias=False)
        
    def forward(self, x):
        ba = - torch.sum((self.linear.weight/2)**2, dim=1)
        ba = ba.float()
#         ba = Variable(ba.float(), requires_grad=False)
        y_hat = self.linear(x) + ba
        y_hat = torch.sigmoid(y_hat)
        return y_hat 

In [22]:
def train_LR(X, Y, learning_rate = 0.01, ep = 100, l2 = False):
    
    model = LgRegv(2048, 5)
    model = model.to(device) 
    X= torch.from_numpy(X).to(device).float()
    Y = torch.from_numpy(Y).to(device).float()
    
    if l2:
        X = X / torch.norm(X.float(), p=2, dim=-1, keepdim=True)
    
    optimizer = torch.optim.Adam(
        model.parameters(), 
        lr=learning_rate, 
        weight_decay=0.01
    )

    loss_function = torch.nn.BCEWithLogitsLoss()
    print('training LR')
    for i in range(1, ep + 1):
        Z = model(X)
        loss = loss_function(Z, Y)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), cfg.clip_max_norm)  
        optimizer.step()
        optimizer.zero_grad()
        
    return model.linear.weight.data.detach().cpu().numpy(), model

In [23]:
def evaluate_fewshot(all_preds, y_trues):
    class_auc = []

    for i in range(all_preds.shape[-1]): 
        if len(np.unique(y_trues[:, i])) <= 1:
            continue
        class_auc.append(roc_auc_score(y_trues[:,i],all_preds[:,i]))
    return class_auc

In [24]:
n_shots = 10
num_episodes = 50
beta=0.5

In [25]:
##RKD  ENSEMBLE
for ns in [50]:
    n_shots = ns
    aucs = []
    only_finetune = []
    rkd = []
    center = False
    l2 = True
    result = defaultdict(list)

    for episode in tqdm(range(num_episodes)):
        gc.collect()
        torch.cuda.empty_cache()

        backbone = deepcopy(model_finetune)
        backbone = backbone.to(device)

        modelrcna = deepcopy(model_rcna)
        modelrcna = modelrcna.to(device)
        
        modelrcna_finetune = deepcopy(model_rcna)
        modelrcna_finetune = modelrcna_finetune.to(device)
        optimizer_nca_finetune = optim.Adam(modelrcna_finetune.parameters(), lr=0.0001, weight_decay=0.0001)
        exp_lr_scheduler_nca_finetune = lr_scheduler.StepLR(optimizer_nca_finetune, step_size=7, gamma=0.1)
        
        model_finetune2 = deepcopy(model_finetune)
        optimizer_finetune = optim.Adam(model_finetune2.parameters(), lr=0.0001, weight_decay=0.0001)
        exp_lr_scheduler_finetune = lr_scheduler.StepLR(optimizer_finetune, step_size=7, gamma=0.1)

        model_finetune2 = model_finetune2.to(device)
        criterion = nn.BCEWithLogitsLoss(reduction='none')

        class_df, support_df, query_df = get_support_query_dfs(
        novel_datasets, [14,15,16,17,18], n_shots=n_shots)
        novel_dataset = get_episode_dataset(class_df, support_df, query_df, dt=dt, sc = [14,15,16,17,18])
        novel_dataloaders, novel_image_datasets = get_episode_dataloaders_no_transform(
        novel_dataset, bs = cfg.batch_size, cfg = cfg)

        model_finetune2 = train_fewshot_model_new(model_finetune2, criterion, optimizer_finetune, exp_lr_scheduler_finetune,
                           num_epochs=25, pth='resnet18_finetuned.pth', dataloaders=novel_dataloaders, verbose=False,
                        network='residual_network', gradient_clipping=gc,
                                                  cfg = cfg
                                       )
        
        modelrcna_finetune = train_fewshot_model_new(modelrcna_finetune, novelNCA, optimizer_nca_finetune,
                                                     exp_lr_scheduler_nca_finetune,
                           num_epochs=25, pth='resnet18_finetuned.pth', dataloaders=novel_dataloaders, verbose=False,
                        network='residual_network', gradient_clipping=gc,
                                                  cfg = cfg
                                       )
        
        ft_clf_auc = np.mean(evaluate(novel_dataloaders['test'], model_finetune2))
        torch.cuda.empty_cache()
        result['ft_clf_auc'].append(ft_clf_auc)
        
        for param in backbone.parameters():
            param.requires_grad = False
        backbone.eval()
        model_finetune2.eval()
        modelrcna.eval()
        modelrcna_finetune.eval()
        with torch.no_grad():
            prototypes = []
            prototypes2 = []
            prototypes_simpleshot = []
            prototypes_rcna = []
            prototypes_rcna_finetune = []
            
            for idx in range(5):
                support_samples = []
                support_samples_simpleshot = []
                support_samples_rcna = []
                support_samples_rcna_finetune = []

                for ix in range(len(novel_image_datasets['train'])):
                    if novel_image_datasets['train'][ix][1][idx] == 1:
                        img_train = novel_image_datasets['train'][ix][0].unsqueeze(0).to(device)
                        out, _ = model_finetune2(img_train)
                        out_backbone, _ = backbone(img_train)
                        out_rcna, _ = modelrcna(img_train)
                        out_rcna_finetune, _ = modelrcna_finetune(img_train)
                        
                        if center:
                            out = (out - train_mean_norm)
                        if l2:
                            out = out / torch.norm(out.float(), p=2, dim=-1, keepdim=True)
                            out_backbone = out_backbone  / torch.norm(out_backbone.float(), p=2, dim=-1, keepdim=True)
                            out_rcna = out_rcna / torch.norm(out_rcna.float(), p=2, dim=-1, keepdim=True)
                            out_rcna_finetune = out_rcna_finetune/torch.norm(out_rcna_finetune.float(), p=2, dim=-1, keepdim=True)
                            
                        support_samples.append(out)
                        support_samples_simpleshot.append(out_backbone)
                        support_samples_rcna.append(out_rcna)
                        support_samples_rcna_finetune.append(out_rcna_finetune)
    
                mean_samples = torch.stack(support_samples).mean(dim=0)
                mean_samples_simpleshot=  torch.stack(support_samples_simpleshot).mean(dim=0)
                mean_samples_rcna = torch.stack(support_samples_rcna).mean(dim=0)
                mean_samples_rcna_finetune = torch.stack(support_samples_rcna_finetune).mean(dim=0)
                
                prototypes.append(mean_samples)
                prototypes_simpleshot.append(mean_samples_simpleshot)
                prototypes_rcna.append(mean_samples_rcna)
                prototypes_rcna_finetune.append(mean_samples_rcna_finetune)
                
            prototypes = torch.stack(prototypes)
            prototypes_simpleshot = torch.stack(prototypes_simpleshot)
            prototypes_rcna = torch.stack(prototypes_rcna)
            prototypes_rcna_finetune = torch.stack(prototypes_rcna_finetune)

            query_samples = []
            query_samples2 = []
            query_samples_simpleshot = []
            query_samples_rcna = []
            query_samples_rcna_finetune = []
            labels_true = []
            for key in range(len(novel_image_datasets['test'])):
                img = novel_image_datasets['test'][key][0].unsqueeze(0).to(device)
                out_backbone, _ = backbone(img)
                query_samples2.append(out_backbone)
                out, _ = model_finetune2(img)
                out_rcna,_ = modelrcna(img)
                out_rcna_finetune,_ = modelrcna_finetune(img)
        
                if center:
                    out = (out - train_mean_norm)
                if l2:
                    out = out / torch.norm(out.float(), p=2, dim=-1, keepdim=True)
                    out_backbone = out_backbone / torch.norm(out_backbone.float(), p=2, dim=-1, keepdim=True)
                    out_rcna = out_rcna / torch.norm(out_rcna.float(), p=2, dim=-1, keepdim=True)
                    out_rcna_finetune = out_rcna_finetune / torch.norm(out_rcna_finetune.float(), p=2, dim=-1, keepdim=True)
                    
                query_samples.append(out)
                query_samples_simpleshot.append(out_backbone)
                query_samples_rcna.append(out_rcna)
                query_samples_rcna_finetune.append(out_rcna_finetune)
                labels_true.append(novel_image_datasets['test'][key][1][-5:])

            query_features = torch.cat([q.unsqueeze(0) for q in query_samples])
            query_features2 = torch.cat([q.unsqueeze(0) for q in query_samples2])
            query_features_rcna = torch.cat([q.unsqueeze(0) for q in query_samples_rcna])
            query_features_DC = torch.pow(query_features2, beta)
            query_features_simpleshot =  torch.cat([q.unsqueeze(0) for q in query_samples_simpleshot])
            query_features_rcna_finetune = torch.cat([q.unsqueeze(0) for q in query_samples_rcna_finetune])
    #         if l2:
    #             query_features2 = query_features2 / torch.norm(query_features2.float(), p=2, dim=-1, keepdim=True)
        support_data = []
        support_label = []
        for inputs, labels, uid in novel_dataloaders['train']:
            inputs = inputs.to(device)
            labels = labels.to(device)

            features, _ = backbone(inputs.to(device))
            support_data.append((features).detach().cpu().numpy())
            support_label.append(labels.detach().cpu().numpy())
        support_data = np.vstack(support_data)
        support_data = np.power(support_data[:, ] ,beta)
        support_label = np.vstack(support_label)

        # ---- distribution calibration and feature sampling
        sampled_data = []
        sampled_label = []
        num_sampled = int(750/n_shots)
        n_ways = 5
        for i in range(n_shots*5):
            mean, cov = distribution_calibration(support_data[i], base_means, base_cov, k=2)
            sampled_data.append(np.random.multivariate_normal(mean=mean, cov=cov, size=num_sampled))
            sampled_label.extend([support_label[i]]*num_sampled)

        n_ways = 5
        sampled_data = np.concatenate([sampled_data[:]]).reshape(n_ways * n_shots * num_sampled, -1)
        X_aug = np.concatenate([support_data, sampled_data])
        Y_aug = np.concatenate([support_label, sampled_label])

        prototypes2, lr_model = train_LR(X_aug, Y_aug, learning_rate=0.001, ep=500, l2 = False)
        prototypes2 = 0.5*prototypes2
        if center:
            prototypes2 = prototypes2 - train_mean_norm4
        prototypes2 = torch.from_numpy(prototypes2).to(device)
    #     if l2:
    #         prototypes2 = prototypes2 / torch.norm(prototypes2.float(), p=2, dim=-1, keepdim=True)

        with torch.no_grad():
            logits = euclidean_metric(query_features.squeeze(1), prototypes.squeeze(1))
            logits_DC = euclidean_metric(query_features_DC.squeeze(1), prototypes2.squeeze(1))
            logits_DC_raw = lr_model(query_features_DC)
            
            logits_simpleshot =  euclidean_metric(query_features_simpleshot.squeeze(1), prototypes_simpleshot.squeeze(1))
            
            logits_rcna = euclidean_metric(query_features_rcna.squeeze(1), prototypes_rcna.squeeze(1))  
            logits_rcna_finetune = euclidean_metric(query_features_rcna_finetune.squeeze(1), prototypes_rcna_finetune.squeeze(1))  
            
            logits_comb = logits + logits_DC + logits_simpleshot + logits_rcna
            logits_comb_rcna_finetune = logits + logits_DC + logits_simpleshot + logits_rcna + logits_rcna_finetune
            
            logits_comb_wo_ft = logits_rcna + logits_DC + logits_simpleshot
            logits_comb_wo_rcna = logits +  logits_DC + logits_simpleshot
            logits_comb_wo_ft = F.softmax(logits_comb_wo_ft, dim=-1)
            logits_comb_wo_rcna = F.softmax(logits_comb_wo_rcna, dim=-1)
            logits_comb = F.softmax(logits_comb, dim=-1)
            logits_comb_rcna_finetune =  F.softmax(logits_comb_rcna_finetune, dim=-1)

            logits_ft = F.softmax(logits, dim=-1)
            logits_simpleshot = F.softmax(logits_simpleshot, dim=-1)
            logits_DC = F.softmax(logits_DC, dim=-1)
            logits_rcna = F.softmax(logits_rcna, dim=-1)
            logits_rcna_finetune = F.softmax(logits_rcna_finetune, dim=-1)
            
            all_preds = logits_comb.cpu().numpy()

            y_trues = torch.cat([l.unsqueeze(0) for l in labels_true])[:,-5:].cpu().numpy()

            y_trues = np.array(y_trues)
            
            result['DC'].append(np.mean(evaluate_fewshot(logits_DC.cpu().numpy(), y_trues)))
            result['FT'].append(np.mean(evaluate_fewshot(logits_ft.cpu().numpy(), y_trues)))
            result['simpleshot'].append(np.mean(evaluate_fewshot(logits_simpleshot.cpu().numpy(), y_trues)))
    
            result['ccvd'].append(np.mean(evaluate_fewshot(all_preds, y_trues)))
            result['ccvd_wo_rcna'].append(np.mean(evaluate_fewshot(logits_comb_wo_rcna.cpu().numpy(), y_trues)))
            result['ccvd_wo_ft'].append(np.mean(evaluate_fewshot(logits_comb_wo_ft.cpu().numpy(), y_trues)))
            result['ccvd_rcna_finetune'].append(np.mean(evaluate_fewshot(logits_comb_rcna_finetune.cpu().numpy(), y_trues)))
            
            result['rcna'].append(np.mean(evaluate_fewshot(logits_rcna.cpu().numpy(), y_trues)))
            result['dc_raw'].append(np.mean(evaluate_fewshot(logits_DC_raw.squeeze(1).cpu().numpy(), y_trues)))
            result['rcna_finetune'].append(np.mean(evaluate_fewshot(logits_rcna_finetune.cpu().numpy(), y_trues)))

        print(np.mean(result['simpleshot']), np.mean(result['FT']), np.mean(result['DC']), np.mean(result['rcna']), 
              np.mean(result['dc_raw']), np.mean(result['rcna_finetune']))
        print(np.mean(result['ccvd_wo_ft']), np.mean(result['ccvd_wo_rcna']),  np.mean(result['ccvd']),
              np.mean(result['ccvd_rcna_finetune']))
        print('---------')
        gc.collect()
        torch.cuda.empty_cache()

    filename = "results_june17_5_" + str(n_shots) + ".csv"
    pd.DataFrame(result).to_csv(filename)

  0%|          | 0/50 [00:00<?, ?it/s]C:\Users\saura\AppData\Local\Temp/ipykernel_2912/1712804310.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  negated_target = torch.tensor(~(target.type(torch.bool)),  dtype=torch.int16)
C:\Users\saura\AppData\Local\Temp/ipykernel_2912/1712804310.py:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  negatives_matrix = torch.tensor(~all_pos.type(torch.bool), dtype=torch.int16).to(device)
C:\Users\saura\AppData\Local\Temp/ipykernel_2912/1712804310.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  

training LR


  2%|▏         | 1/50 [24:40<20:09:22, 1480.88s/it]

0.7120176167395622 0.7111989484517252 0.6975156058619831 0.7015542391749868 0.6271367053214707 0.6292155805582567
0.708461416835439 0.7271065926505044 0.7273552649185524 0.7198112091404802
---------


C:\Users\saura\AppData\Local\Temp/ipykernel_2912/1712804310.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  negated_target = torch.tensor(~(target.type(torch.bool)),  dtype=torch.int16)
C:\Users\saura\AppData\Local\Temp/ipykernel_2912/1712804310.py:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  negatives_matrix = torch.tensor(~all_pos.type(torch.bool), dtype=torch.int16).to(device)
C:\Users\saura\AppData\Local\Temp/ipykernel_2912/1712804310.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(all_pos, dtype=torch.int1

...

C:\Users\saura\AppData\Local\Temp/ipykernel_2912/1712804310.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  negated_target = torch.tensor(~(target.type(torch.bool)),  dtype=torch.int16)
C:\Users\saura\AppData\Local\Temp/ipykernel_2912/1712804310.py:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  negatives_matrix = torch.tensor(~all_pos.type(torch.bool), dtype=torch.int16).to(device)
C:\Users\saura\AppData\Local\Temp/ipykernel_2912/1712804310.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(all_pos, dtype=torch.int1

training LR
0.6997791415393054 0.7063060921386155 0.6937228110309619 0.6805825452649497 0.5928908163601969 0.685058538591495
0.7010188645740936 0.7247369564325775 0.7251392341538762 0.7289241735918243
---------


 98%|█████████▊| 49/50 [20:30:18<24:01, 1441.34s/it]C:\Users\saura\AppData\Local\Temp/ipykernel_2912/1712804310.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  negated_target = torch.tensor(~(target.type(torch.bool)),  dtype=torch.int16)
C:\Users\saura\AppData\Local\Temp/ipykernel_2912/1712804310.py:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  negatives_matrix = torch.tensor(~all_pos.type(torch.bool), dtype=torch.int16).to(device)
C:\Users\saura\AppData\Local\Temp/ipykernel_2912/1712804310.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sou

training LR
0.6996984532339019 0.7066057000779193 0.693573964622543 0.6803487636401587 0.5931316325436229 0.6850806120985163
0.7008343782189576 0.7247500706383495 0.7251422790377039 0.7289216051308312
---------


100%|██████████| 50/50 [20:54:25<00:00, 1505.30s/it]


In [43]:
simpl = [ v for i,v in enumerate(result['simpleshot']) if i %2 == 0]

In [46]:
np.mean(result['simpleshot']), np.mean(simpl)

(0.6716824727656878, 0.6716824727656878)

In [47]:
result['simpleshot'] = simpl

In [58]:
len(result['ft_clf_auc'])

32

In [52]:
filename = "results_june17_4_" + str(n_shots) + ".csv"
pd.DataFrame(result).to_csv(filename)
n_shots

50

In [25]:
evaluate_fewshot(logits_DC_raw.squeeze(1).cpu().numpy(), y_trues)

[0.48, 0.77, 0.33999999999999997, 0.736111111111111, 0.5900000000000001]

In [26]:
##RKD  ENSEMBLE
for ns in [5]:
    n_shots = ns
    aucs = []
    only_finetune = []
    rkd = []
    center = False
    l2 = True
    result = defaultdict(list)

    for episode in tqdm(range(num_episodes)):
        gc.collect()
        torch.cuda.empty_cache()

        backbone = deepcopy(model_finetune)
        backbone = backbone.to(device)

        modelrcna = deepcopy(model_rcna)
        modelrcna = modelrcna.to(device)
        
        modelrcna_finetune = deepcopy(model_rcna)
        modelrcna_finetune = modelrcna_finetune.to(device)
        optimizer_nca_finetune = optim.Adam(modelrcna_finetune.parameters(), lr=0.0001, weight_decay=0.0001)
        exp_lr_scheduler_nca_finetune = lr_scheduler.StepLR(optimizer_nca_finetune, step_size=7, gamma=0.1)
        
        model_finetune2 = deepcopy(model_finetune)
        optimizer_finetune = optim.Adam(model_finetune2.parameters(), lr=0.0001, weight_decay=0.0001)
        exp_lr_scheduler_finetune = lr_scheduler.StepLR(optimizer_finetune, step_size=7, gamma=0.1)

        model_finetune2 = model_finetune2.to(device)
        criterion = nn.BCEWithLogitsLoss(reduction='none')

        class_df, support_df, query_df = get_support_query_dfs(
        novel_datasets, [14,15,16,17,18], n_shots=n_shots)
        novel_dataset = get_episode_dataset(class_df, support_df, query_df, dt=dt, sc = [14,15,16,17,18])
        novel_dataloaders, novel_image_datasets = get_episode_dataloaders_no_transform(
        novel_dataset, bs = cfg.batch_size, cfg = cfg)

        model_finetune2 = train_fewshot_model_new(model_finetune2, criterion, optimizer_finetune, exp_lr_scheduler_finetune,
                           num_epochs=22, pth='resnet18_finetuned.pth', dataloaders=novel_dataloaders, verbose=False,
                        network='residual_network', gradient_clipping=gc,
                                                  cfg = cfg
                                       )
        
        modelrcna_finetune = train_fewshot_model_new(modelrcna_finetune, novelNCA, optimizer_nca_finetune,
                                                     exp_lr_scheduler_nca_finetune,
                           num_epochs=22, pth='resnet18_finetuned.pth', dataloaders=novel_dataloaders, verbose=False,
                        network='residual_network', gradient_clipping=gc,
                                                  cfg = cfg
                                       )
        
        ft_clf_auc = np.mean(evaluate(novel_dataloaders['test'], model_finetune2))
        torch.cuda.empty_cache()
        result['ft_clf_auc'].append(ft_clf_auc)
        
        for param in backbone.parameters():
            param.requires_grad = False
        backbone.eval()
        model_finetune2.eval()
        modelrcna.eval()
        modelrcna_finetune.eval()
        with torch.no_grad():
            prototypes = []
            prototypes2 = []
            prototypes_simpleshot = []
            prototypes_rcna = []
            prototypes_rcna_finetune = []
            
            for idx in range(5):
                support_samples = []
                support_samples_simpleshot = []
                support_samples_rcna = []
                support_samples_rcna_finetune = []

                for ix in range(len(novel_image_datasets['train'])):
                    if novel_image_datasets['train'][ix][1][idx] == 1:
                        img_train = novel_image_datasets['train'][ix][0].unsqueeze(0).to(device)
                        out, _ = model_finetune2(img_train)
                        out_backbone, _ = backbone(img_train)
                        out_rcna, _ = modelrcna(img_train)
                        out_rcna_finetune, _ = modelrcna_finetune(img_train)
                        
                        if center:
                            out = (out - train_mean_norm)
                        if l2:
                            out = out / torch.norm(out.float(), p=2, dim=-1, keepdim=True)
                            out_backbone = out_backbone  / torch.norm(out_backbone.float(), p=2, dim=-1, keepdim=True)
                            out_rcna = out_rcna / torch.norm(out_rcna.float(), p=2, dim=-1, keepdim=True)
                            out_rcna_finetune = out_rcna_finetune/torch.norm(out_rcna_finetune.float(), p=2, dim=-1, keepdim=True)
                            
                        support_samples.append(out)
                        support_samples_simpleshot.append(out_backbone)
                        support_samples_rcna.append(out_rcna)
                        support_samples_rcna_finetune.append(out_rcna_finetune)
    
                mean_samples = torch.stack(support_samples).mean(dim=0)
                mean_samples_simpleshot=  torch.stack(support_samples_simpleshot).mean(dim=0)
                mean_samples_rcna = torch.stack(support_samples_rcna).mean(dim=0)
                mean_samples_rcna_finetune = torch.stack(support_samples_rcna_finetune).mean(dim=0)
                
                prototypes.append(mean_samples)
                prototypes_simpleshot.append(mean_samples_simpleshot)
                prototypes_rcna.append(mean_samples_rcna)
                prototypes_rcna_finetune.append(mean_samples_rcna_finetune)
                
            prototypes = torch.stack(prototypes)
            prototypes_simpleshot = torch.stack(prototypes_simpleshot)
            prototypes_rcna = torch.stack(prototypes_rcna)
            prototypes_rcna_finetune = torch.stack(prototypes_rcna_finetune)

            query_samples = []
            query_samples2 = []
            query_samples_simpleshot = []
            query_samples_rcna = []
            query_samples_rcna_finetune = []
            labels_true = []
            for key in range(len(novel_image_datasets['test'])):
                img = novel_image_datasets['test'][key][0].unsqueeze(0).to(device)
                out_backbone, _ = backbone(img)
                query_samples2.append(out_backbone)
                out, _ = model_finetune2(img)
                out_rcna,_ = modelrcna(img)
                out_rcna_finetune,_ = modelrcna_finetune(img)
        
                if center:
                    out = (out - train_mean_norm)
                if l2:
                    out = out / torch.norm(out.float(), p=2, dim=-1, keepdim=True)
                    out_backbone = out_backbone / torch.norm(out_backbone.float(), p=2, dim=-1, keepdim=True)
                    out_rcna = out_rcna / torch.norm(out_rcna.float(), p=2, dim=-1, keepdim=True)
                    out_rcna_finetune = out_rcna_finetune / torch.norm(out_rcna_finetune.float(), p=2, dim=-1, keepdim=True)
                    
                query_samples.append(out)
                query_samples_simpleshot.append(out_backbone)
                query_samples_rcna.append(out_rcna)
                query_samples_rcna_finetune.append(out_rcna_finetune)
                labels_true.append(novel_image_datasets['test'][key][1][-5:])

            query_features = torch.cat([q.unsqueeze(0) for q in query_samples])
            query_features2 = torch.cat([q.unsqueeze(0) for q in query_samples2])
            query_features_rcna = torch.cat([q.unsqueeze(0) for q in query_samples_rcna])
            query_features_DC = torch.pow(query_features2, beta)
            query_features_simpleshot =  torch.cat([q.unsqueeze(0) for q in query_samples_simpleshot])
            query_features_rcna_finetune = torch.cat([q.unsqueeze(0) for q in query_samples_rcna_finetune])
    #         if l2:
    #             query_features2 = query_features2 / torch.norm(query_features2.float(), p=2, dim=-1, keepdim=True)
        support_data = []
        support_label = []
        for inputs, labels, uid in novel_dataloaders['train']:
            inputs = inputs.to(device)
            labels = labels.to(device)

            features, _ = backbone(inputs.to(device))
            support_data.append((features).detach().cpu().numpy())
            support_label.append(labels.detach().cpu().numpy())
        support_data = np.vstack(support_data)
        support_data = np.power(support_data[:, ] ,beta)
        support_label = np.vstack(support_label)

        # ---- distribution calibration and feature sampling
        sampled_data = []
        sampled_label = []
        num_sampled = int(750/n_shots)
        n_ways = 5
        for i in range(n_shots*5):
            mean, cov = distribution_calibration(support_data[i], base_means, base_cov, k=2)
            sampled_data.append(np.random.multivariate_normal(mean=mean, cov=cov, size=num_sampled))
            sampled_label.extend([support_label[i]]*num_sampled)

        n_ways = 5
        sampled_data = np.concatenate([sampled_data[:]]).reshape(n_ways * n_shots * num_sampled, -1)
        X_aug = np.concatenate([support_data, sampled_data])
        Y_aug = np.concatenate([support_label, sampled_label])

        prototypes2, lr_model = train_LR(X_aug, Y_aug, learning_rate=0.001, ep=400, l2 = False)
        prototypes2 = 0.5*prototypes2
        if center:
            prototypes2 = prototypes2 - train_mean_norm4
        prototypes2 = torch.from_numpy(prototypes2).to(device)
    #     if l2:
    #         prototypes2 = prototypes2 / torch.norm(prototypes2.float(), p=2, dim=-1, keepdim=True)

        with torch.no_grad():
            logits = euclidean_metric(query_features.squeeze(1), prototypes.squeeze(1))
            logits_DC = euclidean_metric(query_features_DC.squeeze(1), prototypes2.squeeze(1))
            logits_DC_raw = lr_model(query_features_DC)
            
            logits_simpleshot =  euclidean_metric(query_features_simpleshot.squeeze(1), prototypes_simpleshot.squeeze(1))
            
            logits_rcna = euclidean_metric(query_features_rcna.squeeze(1), prototypes_rcna.squeeze(1))  
            logits_rcna_finetune = euclidean_metric(query_features_rcna_finetune.squeeze(1), prototypes_rcna_finetune.squeeze(1))  
            
            logits_comb = logits + logits_DC + logits_simpleshot + logits_rcna
            logits_comb_rcna_finetune = logits + logits_DC + logits_simpleshot + logits_rcna + logits_rcna_finetune
            
            logits_comb_wo_ft = logits_rcna + logits_DC + logits_simpleshot
            logits_comb_wo_rcna = logits +  logits_DC + logits_simpleshot
            logits_comb_wo_ft = F.softmax(logits_comb_wo_ft, dim=-1)
            logits_comb_wo_rcna = F.softmax(logits_comb_wo_rcna, dim=-1)
            logits_comb = F.softmax(logits_comb, dim=-1)
            logits_comb_rcna_finetune =  F.softmax(logits_comb_rcna_finetune, dim=-1)

            logits_ft = F.softmax(logits, dim=-1)
            logits_simpleshot = F.softmax(logits_simpleshot, dim=-1)
            logits_DC = F.softmax(logits_DC, dim=-1)
            logits_rcna = F.softmax(logits_rcna, dim=-1)
            logits_rcna_finetune = F.softmax(logits_rcna_finetune, dim=-1)
            
            all_preds = logits_comb.cpu().numpy()

            y_trues = torch.cat([l.unsqueeze(0) for l in labels_true])[:,-5:].cpu().numpy()

            y_trues = np.array(y_trues)
            
            result['DC'].append(np.mean(evaluate_fewshot(logits_DC.cpu().numpy(), y_trues)))
            result['FT'].append(np.mean(evaluate_fewshot(logits_ft.cpu().numpy(), y_trues)))
            result['simpleshot'].append(np.mean(evaluate_fewshot(logits_simpleshot.cpu().numpy(), y_trues)))
    
            result['ccvd'].append(np.mean(evaluate_fewshot(all_preds, y_trues)))
            result['ccvd_wo_rcna'].append(np.mean(evaluate_fewshot(logits_comb_wo_rcna.cpu().numpy(), y_trues)))
            result['ccvd_wo_ft'].append(np.mean(evaluate_fewshot(logits_comb_wo_ft.cpu().numpy(), y_trues)))
            result['ccvd_rcna_finetune'].append(np.mean(evaluate_fewshot(logits_comb_rcna_finetune.cpu().numpy(), y_trues)))
            
            result['rcna'].append(np.mean(evaluate_fewshot(logits_rcna.cpu().numpy(), y_trues)))
            result['dc_raw'].append(np.mean(evaluate_fewshot(logits_DC_raw.squeeze(1).cpu().numpy(), y_trues)))
            result['rcna_finetune'].append(np.mean(evaluate_fewshot(logits_rcna_finetune.cpu().numpy(), y_trues)))

        print(np.mean(result['simpleshot']), np.mean(result['FT']), np.mean(result['DC']), np.mean(result['rcna']), 
              np.mean(result['dc_raw']), np.mean(result['rcna_finetune']))
        print(np.mean(result['ccvd_wo_ft']), np.mean(result['ccvd_wo_rcna']),  np.mean(result['ccvd']),
              np.mean(result['ccvd_rcna_finetune']))
        print('---------')
        gc.collect()
        torch.cuda.empty_cache()

    filename = "results_june17_4_" + str(n_shots) + ".csv"
    pd.DataFrame(result).to_csv(filename)

  0%|          | 0/50 [00:00<?, ?it/s]C:\Users\saura\AppData\Local\Temp/ipykernel_2912/1712804310.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  negated_target = torch.tensor(~(target.type(torch.bool)),  dtype=torch.int16)
C:\Users\saura\AppData\Local\Temp/ipykernel_2912/1712804310.py:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  negatives_matrix = torch.tensor(~all_pos.type(torch.bool), dtype=torch.int16).to(device)
C:\Users\saura\AppData\Local\Temp/ipykernel_2912/1712804310.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  

training LR


  2%|▏         | 1/50 [04:29<3:40:02, 269.43s/it]

0.5596893704850361 0.5324220846233231 0.5566718266253871 0.6065624355005159 0.4991805985552116 0.5868771929824562
0.5609938080495356 0.5387543859649123 0.5480175438596492 0.5501733746130031
---------


C:\Users\saura\AppData\Local\Temp/ipykernel_2912/1712804310.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  negated_target = torch.tensor(~(target.type(torch.bool)),  dtype=torch.int16)
C:\Users\saura\AppData\Local\Temp/ipykernel_2912/1712804310.py:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  negatives_matrix = torch.tensor(~all_pos.type(torch.bool), dtype=torch.int16).to(device)
C:\Users\saura\AppData\Local\Temp/ipykernel_2912/1712804310.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(all_pos, dtype=torch.int1

...

0.6244111278596651 0.605012725892362 0.6273518916338465 0.6039984635458193 0.5801703764212451 0.5997306135486776
0.6294658244250081 0.633232691071416 0.6338201526224525 0.6365760830648048
---------


C:\Users\saura\AppData\Local\Temp/ipykernel_2912/1712804310.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  negated_target = torch.tensor(~(target.type(torch.bool)),  dtype=torch.int16)
C:\Users\saura\AppData\Local\Temp/ipykernel_2912/1712804310.py:38: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  negatives_matrix = torch.tensor(~all_pos.type(torch.bool), dtype=torch.int16).to(device)
C:\Users\saura\AppData\Local\Temp/ipykernel_2912/1712804310.py:42: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(all_pos, dtype=torch.int1

training LR


100%|██████████| 50/50 [3:31:17<00:00, 253.56s/it]

0.6247411145510836 0.6056154248366012 0.6266574863629663 0.6050211666420955 0.5801074436090226 0.6009876844070962
0.6294062813897489 0.6331899076121676 0.6338267777286353 0.6368370583321047
---------


In [23]:
pd.DataFrame(result).to_csv('5_new.csv')

In [25]:
filename = "ccvd_" + str(n_shots) + ".csv"

In [27]:
filename

'ccvd_5.csv'

In [ ]:
from torch.autograd import Variable
